In [1]:
!pip install dagshub mlflow --quiet

import warnings
warnings.filterwarnings("ignore")

print("Done!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 60.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [10]:
import mlflow.sklearn
from datetime import datetime
import joblib
import dagshub
import mlflow

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

Initialized MLflow to track repo "gnada22/ml_final_project"

Repository gnada22/ml_final_project initialized!

In [11]:
local_path = mlflow.artifacts.download_artifacts(
    artifact_uri="mlflow-artifacts:/bdaee488c065473c96d054e5ae43539a/9e4a788fee9148d6b07316e4ccbc0edd/artifacts/model.pkl"
)

model = joblib.load(local_path)

print("Done!")

Done!


In [12]:
# load and add custom features

test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip", parse_dates=["Date"])
features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = test.merge(features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(stores, on="Store", how="left")

def add_date_features(df):
    df["weekofyear"] = df["Date"].dt.isocalendar().week.astype(int)
    df["sin_13"] = np.sin(2 * np.pi * df["weekofyear"] / 13)
    df["cos_13"] = np.cos(2 * np.pi * df["weekofyear"] / 13)
    df["sin_23"] = np.sin(2 * np.pi * df["weekofyear"] / 23)
    df["cos_23"] = np.cos(2 * np.pi * df["weekofyear"] / 23)
    return df

df = add_date_features(df)

train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
train = train[["Store", "Dept", "Date", "Weekly_Sales"]]
train = train.sort_values(["Store", "Dept", "Date"])

full = pd.concat([train, df], axis=0)
full = full.sort_values(["Store", "Dept", "Date"])

full["lag_1"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
full["lag_52"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)

df = full[full["Weekly_Sales"].isna()].copy()  # just test rows
# df.dropna(subset=["lag_1", "lag_52"], inplace=True)  # drop rows with missing lags

categorical = ["Store", "Dept", "Type"]
numerical = ["Temperature", "Fuel_Price", "CPI", "Unemployment",
             "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]
engineered = ["lag_1", "lag_52", "weekofyear", "sin_13", "cos_13", "sin_23", "cos_23"]

features = categorical + numerical + engineered

X_test = df[features].copy()

In [13]:
preds = model.predict(X_test)
df["Weekly_Sales"] = preds

df["Id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str) + "_" + df["Date"].dt.strftime("%Y-%m-%d")
submission = df[["Id", "Weekly_Sales"]]
submission.to_csv("submission.csv", index=False)
print("✅ Submission saved as submission.csv")

✅ Submission saved as submission.csv
